In [ ]:
!nvidia-smi

In [ ]:
import torch
from pathlib import Path

In [ ]:
from model.models import SimpleCNN4Layer_D35_sp as Model
from model.collectdata import collect_data
from model.plots import plot_truth_vs_predict
from model.efficiency import efficiency, ValueSet
from model.training import select_gpu
from model.core import modernize

#### Device configuration

In [ ]:
device = select_gpu(2)
#device = torch.device("cpu")

#### Load a dataset

In [ ]:
valfile = Path('/share/lazy/schreihf/PvFinder/Oct03_20K_val.h5')
validation = collect_data(valfile,
                          batch_size=1,
                          device=device,
                          slice=slice(10000))
                          

In [ ]:
model = Model().to(device)

#### Load the model weights

Modernizing old style models if needed.

In [ ]:
state = torch.load('/share/lazy/schreihf/PvFinder/models/Dec11_SimpleCNN4Layer_D35_sp_first200epochs_240K_lr_3em5_bs512_Alt_Loss_A_4p5_final.pyt')
state = modernize(state, 3)
model.load_state_dict(state)
model.eval()

#### Compute the model and move results to Numpy

In [ ]:
%%time
with torch.no_grad():
    outputs = model(validation.dataset.tensors[0]).cpu().numpy()
    labels = validation.dataset.tensors[1].cpu().numpy()

#### Compute the efficencies over the dataset

In [ ]:
%%time
total = ValueSet(0,0,0,0)

for label, output in zip(labels, outputs):
    total += efficiency(label, output, 5., 1e-2, .2, 3)

In [ ]:
print(total)
print()
print(total.pretty())

#### Some examples of the counting algorithm:

In [ ]:
for i in range(20):
    result = efficiency(labels[i], outputs[i], 5., 1e-2, .2, 3)
    print(result)

In [ ]:
for i in range(20):
    result = efficiency(labels[i], outputs[i], 5., 1e-2,.2, 3)
    ax = plot_truth_vs_predict(labels[i], outputs[i])
    ax.set_title(str(result))